# Data Cleanup & Additionnal Metrics

This notebook contains all the steps necessary to associate classes and methods in UND data collected before and to identify files with bugs. 

## 1. Processing UND Data

First, we will get rid of the all the classes and methods in each entry into the CSV and a 'Bug' column initialized at 0.

In [1]:
import os
import glob
import csv
import pandas as pd
from pathlib import Path as path

In order to simplify repertory changes, we'll initialize two variables, containning the paths of this current repository and the path of your clone of the Apache Hive repertory.

In [2]:
project_repo = path("/home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/")
hive_repo = path("/home/nicolas-richard/Desktop/.Apache_Hive/")

In [3]:
def process_csv(file_path):
    df = pd.read_csv(file_path)
    
    df = df[df['Kind'] == 'File']

    df = df.drop(columns=['Kind','Entity_Uniquename'])  
    df.insert(0, 'Bug', 0)  
    
    df = df.rename(columns={'Name': 'FileName'})
    
    columns_order = ['Bug', 'FileName'] + [col for col in df.columns if col not in ['Bug', 'FileName']]
    df = df[columns_order]
    
    return df

current_repo = os.getcwd()
input_files = glob.glob(os.path.join(current_repo, 'UND_hive_data', '*.csv'))
output_dir = os.path.join(current_repo, 'UND_hive_processed_data')

os.makedirs(output_dir, exist_ok=True)
output_files = []

for file_path in input_files:
    processed_df = process_csv(file_path)

    base_name = os.path.basename(file_path) 
    output_file = os.path.join(output_dir, base_name.replace('.csv', '_processed.csv'))
    processed_df.to_csv(output_file, index=False)
    output_files.append(output_file)

print("Processing complete. Files saved:", output_files)

Processing complete. Files saved: ['/home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_processed_data/UND_hive-2.3.8_processed.csv', '/home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_processed_data/UND_hive-2.2.0_processed.csv', '/home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_processed_data/UND_hive-4.0.0_processed.csv', '/home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_processed_data/UND_hive-2.3.3_processed.csv', '/home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_processed_data/UND_hive-3.1.3_processed.csv', '/home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_processed_data/UND_hive-2.3.7_processed.csv', '/home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_processed_data/UND_hive-2.3.10_processed.csv', '/home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_processed_data/UND_hive-2.

## 2. Identify Files with Bugs

For each of the affected files identified in the previous notebook, we will initialize at '1' the bug column 

In [4]:
os.chdir(project_repo)
current_repo = os.getcwd()
input_file = 'Hive_Modified_Files.csv'

bug_ids = []
versions = []
file_names = []

def normalize_path(path):
    return os.path.normpath(path).lower()

with open(input_file, 'r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    headers = next(reader, None)
    for row in reader:
        if len(row) < 3:
            continue
        bug_id = row[0].strip()
        version = row[1].strip()
        affected_files = row[2].split(';')
        for affected_file in affected_files:
            affected_file = affected_file.strip()
            if affected_file:  # Ensure the file name is not empty
                bug_ids.append(bug_id)
                versions.append(version)
                file_names.append(affected_file)

output_df = pd.DataFrame({
    'Bug ID': bug_ids,
    'Version': versions,
    'File': file_names
})

print("Parsed data from input file:")
print(output_df)

unfound_bug_ids = []
unfound_versions = []
unfound_file_names = []

found_files = 0
unfound_files = 0

for index, row in output_df.iterrows():
    version = row['Version']
    target_file = row['File']
    filename = f"{current_repo}/UND_hive_processed_data/UND_hive-{version}_processed.csv"

    if not os.path.isfile(filename):
        print(f"File not found: {filename}")
        unfound_bug_ids.append(row['Bug ID'])
        unfound_versions.append(version)
        unfound_file_names.append(target_file)
        unfound_files += 1
        continue

    try:
        with open(filename, 'r', newline='', encoding='utf-8') as file:
            rows = list(csv.reader(file))

        found = False
        target_file_lower = normalize_path(target_file)

        for i, columns in enumerate(rows):
            if len(columns) < 2:
                continue
            file_name_in_row_lower = normalize_path(columns[1])

            matches = [
                target_file_lower in file_name_in_row_lower,
                file_name_in_row_lower in target_file_lower,
                target_file_lower.replace('/', '\\') in file_name_in_row_lower,
                file_name_in_row_lower.replace('/', '\\') in target_file_lower,
                target_file_lower.split('.')[0] in file_name_in_row_lower,
                file_name_in_row_lower.split('.')[0] in target_file_lower
            ]

            if any(matches):
                columns[0] = '1'  
                rows[i] = columns
                found = True
                found_files += 1
                print(f"Match found for Bug {row['Bug ID']} in {row['Version']}")
                break

        if not found:
            unfound_files += 1
            unfound_bug_ids.append(row['Bug ID'])
            unfound_versions.append(version)
            unfound_file_names.append(target_file)
            print(f"Match not found for Bug {row['Bug ID']} in {row['Version']}")
        else:
            with open(filename, 'w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerows(rows)

    except Exception as e:
        print(f"Error processing file {filename}: {e}")
        unfound_bug_ids.append(row['Bug ID'])
        unfound_versions.append(version)
        unfound_file_names.append(target_file)

unfound_df = pd.DataFrame({
    'Bug ID': unfound_bug_ids,
    'Version': unfound_versions,
    'File': unfound_file_names
})

unfound_csv_path = 'Unfound_Files.csv'
unfound_df.to_csv(unfound_csv_path, index=False)
print(f"Unfound files saved to {unfound_csv_path}")

print(f"Total files found: {found_files}")
print(f"Total files not found: {unfound_files}")
print(f"Percentage of files found: {found_files/(unfound_files+found_files) * 100}%")

Parsed data from input file:
          Bug ID Version                                               File
0     HIVE-22165   2.1.0  service/src/java/org/apache/hive/service/cli/s...
1     HIVE-21009   2.1.0  common/src/java/org/apache/hadoop/hive/conf/Hi...
2     HIVE-21009   2.1.0  service/src/java/org/apache/hive/service/auth/...
3     HIVE-21009   2.1.0  service/src/test/org/apache/hive/service/auth/...
4     HIVE-20771   3.1.0  serde/src/java/org/apache/hadoop/hive/serde2/l...
...          ...     ...                                                ...
4495  HIVE-21614   2.3.4  standalone-metastore/metastore-server/src/main...
4496  HIVE-21614   2.3.4  standalone-metastore/metastore-server/src/main...
4497  HIVE-21508   2.3.4  standalone-metastore/metastore-common/src/main...
4498  HIVE-16839   2.3.4  standalone-metastore/metastore-server/src/main...
4499  HIVE-16839   2.3.4  standalone-metastore/metastore-server/src/test...

[4500 rows x 3 columns]
Match found for Bug HIVE-22165 in 

## 3. Add Classes and Methods to Processed Files

As mentionned before, within the UND data collected, only files have been identified as containning a bug. UND also provides variables for classes and methods, which we will need to associate to their respective file.  

### 3.1 Classes
We want to come up with values for the following fields for a given file, from the  
- CountClassBase
- CountClassCoupled
- CountClassDerived
- MaxInheritanceTree
- CountInputMin
- CountInputMean
- CountInputMax
- CountOutputMin
- CountOutputMean
- CountOutputMax
- CountPathMin
- CountPathMean
- CountPathMax
- MaxNestingMin
- MaxNestingMean
- MaxNestingMax

To get the first three metrics, we'll simply add the ClassBaseCount, CountClassDerived and CountClassCoupled metrics from each of the classes linked to a given file. For the last metric, we'll select maximum value of MaxInheritanceTree for each class linked to the file.
For the metrics related to methods, we'll select only "methods" in the original UND dataset and create new columns for each of the variables above. For each set of variables, we'll simply select the minimal, average and maimal values for each method.

In [5]:
def process_file(processed_file, processed_data_dir, data_dir, output_dir):
    if not processed_file.endswith('_processed.csv'):
        return

    try:
        version = processed_file.split('_processed.csv')[0].split('UND_hive-')[1]
    except IndexError:
        print(f"Filename {processed_file} does not match the expected format. Skipping.")
        return

    data_file = f'UND_hive-{version}.csv'
    data_file_path = os.path.join(data_dir, data_file)

    if not os.path.isfile(data_file_path):
        print(f"Data file {data_file} not found in {data_dir}. Skipping {processed_file}.")
        return

    try:
        processed_df = pd.read_csv(os.path.join(processed_data_dir, processed_file))
    except Exception as e:
        print(f"Error reading {processed_file}: {e}. Skipping.")
        return

    try:
        data_df = pd.read_csv(data_file_path)
    except Exception as e:
        print(f"Error reading {data_file}: {e}. Skipping.")
        return

    kind_col = data_df.columns[0]

    filename_col_processed = processed_df.columns[1]  

    possible_filename_cols = ['FilePath', 'FileName', 'File', 'filename']
    filename_col_data = None
    for col in possible_filename_cols:
        if col in data_df.columns:
            filename_col_data = col
            break
    if not filename_col_data:
        print(f"No matching filename column found in {data_file}. Skipping {processed_file}.")
        return

    # Process method entries
    method_entries = data_df[data_df[kind_col].str.contains('method', case=False, na=False)]

    method_required_metrics = ['CountInput', 'CountOutput', 'CountPath', 'MaxNesting']
    method_missing_metrics = [metric for metric in method_required_metrics if metric not in data_df.columns]
    if method_missing_metrics:
        print(f"Missing method metric columns {method_missing_metrics} in {data_file}. Skipping method metrics.")
    else:
        try:
            method_aggregated_metrics = method_entries.groupby(filename_col_data).agg({
                'CountInput': ['min', 'mean', 'max'],
                'CountOutput': ['min', 'mean', 'max'],
                'CountPath': ['min', 'mean', 'max'],
                'MaxNesting': ['min', 'mean', 'max']
            }).reset_index()
        except Exception as e:
            print(f"Error aggregating method metrics for {processed_file}: {e}. Skipping method metrics.")
            method_aggregated_metrics = None

        if method_aggregated_metrics is not None:
            method_aggregated_metrics.columns = [
                f"{col[0]}{col[1].capitalize()}" if col[1] else col[0]
                for col in method_aggregated_metrics.columns
            ]
            method_new_metric_columns = [
                'CountInputMin', 'CountInputMean', 'CountInputMax',
                'CountOutputMin', 'CountOutputMean', 'CountOutputMax',
                'CountPathMin', 'CountPathMean', 'CountPathMax',
                'MaxNestingMin', 'MaxNestingMean', 'MaxNestingMax'
            ]

            processed_df = processed_df.merge(
                method_aggregated_metrics,
                how='left',
                left_on=filename_col_processed,
                right_on=filename_col_data,
                suffixes=('', '_method')
            )

            for metric in method_new_metric_columns:
                if metric in processed_df.columns:
                    processed_df[metric] = processed_df[metric].fillna(0)
                else:
                    processed_df[metric] = 0

    class_entries = data_df[data_df[kind_col].str.contains('class', case=False, na=False)]

    class_required_metrics = ['CountClassBase', 'CountClassCoupled', 'CountClassDerived', 'MaxInheritanceTree','PercentLackOfCohesion']
    class_missing_metrics = [metric for metric in class_required_metrics if metric not in data_df.columns]
    if class_missing_metrics:
        print(f"Missing class metric columns {class_missing_metrics} in {data_file}. Skipping class metrics.")
    else:
        try:
            class_aggregated_metrics = class_entries.groupby(filename_col_data).agg({
                'CountClassBase': 'sum',
                'CountClassCoupled': 'sum',
                'CountClassDerived': 'sum',
                'MaxInheritanceTree': 'max',
                'PercentLackOfCohesion': 'mean'
            }).reset_index()
        except Exception as e:
            print(f"Error aggregating class metrics for {processed_file}: {e}. Skipping class metrics.")
            class_aggregated_metrics = None

        if class_aggregated_metrics is not None:
            processed_df = processed_df.merge(
                class_aggregated_metrics,
                how='left',
                left_on=filename_col_processed,
                right_on=filename_col_data,
                suffixes=('', '_class')
            )

            for metric in class_required_metrics:
                if metric in processed_df.columns:
                    processed_df[metric] = processed_df[metric].fillna(0)
                else:
                    processed_df[metric] = 0

    if filename_col_data in processed_df.columns:
        processed_df.drop(columns=[filename_col_data], inplace=True)

    output_file_path = os.path.join(output_dir, processed_file)

    try:
        processed_df.to_csv(output_file_path, index=False)
        print(f"Successfully updated {processed_file} and saved to {output_dir}.")
    except Exception as e:
        print(f"Error saving updated file {output_file_path}: {e}.")
        return

def process_files(processed_data_dir, data_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for processed_file in os.listdir(processed_data_dir):
        process_file(processed_file, processed_data_dir, data_dir, output_dir)

if __name__ == "__main__":
    
    processed_data_directory = project_repo.joinpath('UND_hive_processed_data')
    data_directory = project_repo.joinpath('UND_hive_data')
    output_directory = project_repo.joinpath('UND_hive_updated_data')

    process_files(processed_data_directory, data_directory, output_directory)

Successfully updated UND_hive-2.3.10_processed.csv and saved to /home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_updated_data.
Successfully updated UND_hive-2.3.2_processed.csv and saved to /home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_updated_data.
Successfully updated UND_hive-2.3.5_processed.csv and saved to /home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_updated_data.
Successfully updated UND_hive-4.0.1_processed.csv and saved to /home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_updated_data.
Successfully updated UND_hive-2.3.1_processed.csv and saved to /home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_updated_data.
Successfully updated UND_hive-3.1.2_processed.csv and saved to /home/nicolas-richard/Desktop/.Apache_Hive_Bug_Prediction_ML_Model/UND_hive_updated_data.
Successfully updated UND_hive-2.1.0_processed.csv and saved to /home/nicolas-rich

## 4. Additionnal Metrics
As per the specification of the personnal project, we'll try and gather additionnal metrics to improve our model. 

### 4.1 Lines added & deleted from a given version

We'll begin by creating a dictionnary of versions and corresponding commits from the file `Hive_Last_Commits.csv`previously created

In [34]:
last_commits = open(os.path.join(project_repo,"Hive_Last_Commits.csv"), "r")

versions = []
commits_by_version = {}

for line in last_commits.readlines():
    parts = line.strip().split(": ")
    version = parts[0].split("-")[1]  
    commit = parts[-1].split(" ")[0]  
    versions.append(str(version))
    commits_by_version[version] = commit
    
last_commits.close()

print(commits_by_version)
print(versions)

{'2.0.0': '21cf6ff3789ed94bdf61587e2e73fb94b1d9304c', '2.0.1': '115d225d8d311e1d1fa42e8f103c214b26c209a2', '2.1.0': 'c168af26de36ecb66e606b040c862a222ee2a190', '2.1.1': '52f1b2471545a797856e4b9b1ae0a36cb4233c18', '2.2.0': '52f1b2471545a797856e4b9b1ae0a36cb4233c18', '2.3.0': '90149de71eecb3374fcee0b876324a3b84b8a93d', '2.3.1': 'd7d96658c233b100754884985860bf40f97264ee', '2.3.2': 'b2e411e8cad0da0af370e2b3e2d0bf9cb32dfe7f', '2.3.3': '52f1b2471545a797856e4b9b1ae0a36cb4233c18', '2.3.4': '353c55e22d0b0f4603053a7ef45afb5b7b244b75', '2.3.5': 'de69a0f9d973822895f3ff0744ee3af768118268', '2.3.6': 'db59ec6ecf57dad72588b7593fb516c3e810b5a2', '2.3.7': '03efbb3db1bbccfd3548fa7fa3f130be081e1cdd', '2.3.8': '89073a94354f0cc14ec4ae0a43e05aae29276b4d', '2.3.9': '01b8fcffcb31cc128e71350d1b25c98b50ee85e5', '2.3.10': '7950967eae9640fcc0aa22f4b6c7906b34281eac', '3.0.0': 'ed95ee90608a73fe08747b5483f1bc5ff14bfa22', '3.1.0': '65f02d2f99b990cb28fd6a832fa3425042e60a04', '3.1.1': '4b40f7d2b35df26af773ec138fdf521231

In [ ]:
import os
import pandas as pd

csv_files = sorted([f for f in os.listdir(output_directory) if f.endswith('_processed.csv')])

for file in csv_files:
    
    df = pd.read_csv(os.path.join(output_directory, file))
    version = file.split("-")[-1].split("_")[0]

    print(f"\n=== Processing version : {version} ===")

    for another_file in csv_files:
        another_version = another_file.split("-")[-1].split("_")[0]
        if another_version >= version:
            continue  

        if f"LinesAddedSince{another_version}" not in df.columns:
            df[f"LinesAddedSince{another_version}"] = 0
        if f"LinesRemovedSince{another_version}" not in df.columns:
            df[f"LinesRemovedSince{another_version}"] = 0

    for another_file in csv_files:
        another_version = another_file.split("-")[-1].split("_")[0]
        if another_version >= version or another_version == version:
            continue  

        print(f"  Comparing with earlier version: {another_version}")
        another_df = pd.read_csv(os.path.join(output_directory, another_file))

        for index, row in df.iterrows():
            file_name = row["FileName"]
            line_count = row["CountLine"]

            matching_rows = another_df[another_df["FileName"] == file_name]

            if not matching_rows.empty:
                another_line_count = matching_rows.iloc[0]["CountLine"]

                print(f"    - {file_name} found in version {another_version}")

                if line_count > another_line_count:
                    added_lines = line_count - another_line_count
                    df.loc[index, f"LinesAddedSince{another_version}"] = added_lines
                    print(f"      Lines added: {added_lines}")
                elif line_count < another_line_count:
                    removed_lines = another_line_count - line_count
                    df.loc[index, f"LinesRemovedSince{another_version}"] = removed_lines
                    print(f"      Lines removed: {removed_lines}")

    output_path = os.path.join(output_directory, file)
    df.to_csv(output_path, index=False)
    print(f"=== Updated file saved as {output_path} ===\n")



=== Processing version : 2.0.0 ===
  Comparing with earlier version: 2.0.0
    - /home/nicolas-richard/Desktop/.Apache_Hive/accumulo-handler/src/java/org/apache/hadoop/hive/accumulo/AccumuloConnectionParameters.java found in version 2.0.0
    - /home/nicolas-richard/Desktop/.Apache_Hive/accumulo-handler/src/java/org/apache/hadoop/hive/accumulo/AccumuloHiveConstants.java found in version 2.0.0
    - /home/nicolas-richard/Desktop/.Apache_Hive/accumulo-handler/src/java/org/apache/hadoop/hive/accumulo/AccumuloHiveRow.java found in version 2.0.0
    - /home/nicolas-richard/Desktop/.Apache_Hive/accumulo-handler/src/java/org/apache/hadoop/hive/accumulo/AccumuloStorageHandler.java found in version 2.0.0
    - /home/nicolas-richard/Desktop/.Apache_Hive/accumulo-handler/src/java/org/apache/hadoop/hive/accumulo/HiveAccumuloHelper.java found in version 2.0.0
    - /home/nicolas-richard/Desktop/.Apache_Hive/accumulo-handler/src/java/org/apache/hadoop/hive/accumulo/LazyAccumuloMap.java found in ver

KeyboardInterrupt: 